# CMIP6 Compute GSL

**Following steps are included in this script:**

1. Load netCDF files
2. Compute WUE

Save and replace netcdf files

In [50]:
# ========== Packages ==========
import xarray as xr
import pandas as pd
import numpy as np
import dask
import os
import glob
import copy

import matplotlib.pyplot as plt

%matplotlib inline

### Functions

In [51]:
def select_period(ds_dict, start_year=None, end_year=None, period=None, yearly_sum=False):
    '''
    Helper function to select periods and optionally compute yearly sums.
    
    Parameters:
    ds_dict (dict): Dictionary with xarray datasets.
    start_year (int): The start year of the period.
    end_year (int): The end year of the period.
    period (int, list, str, None): Single month (int), list of months (list), multiple seasons (str) to select,
                                   or None to not select any specific period.
    yearly_sum (bool): If True, compute the yearly sum over the selected period.
    '''
    
    # Create a deep copy of the original ds_dict to avoid modifying it directly
    ds_dict_copy = copy.deepcopy(ds_dict)

    # Define season to month mapping for northern hemisphere
    seasons_to_months = {
        'nh_winter': [12, 1, 2],
        'nh_spring': [3, 4, 5],
        'nh_summer': [6, 7, 8],
        'nh_fall': [9, 10, 11]
    }
    
    # Define month name mapping
    month_names = {
        1: 'J', 2: 'F', 3: 'M', 4: 'A', 5: 'M', 6: 'J',
        7: 'J', 8: 'A', 9: 'S', 10: 'O', 11: 'N', 12: 'D'
    }

    # Define number of days per month (assuming 28 days for February)
    days_per_month = {
        1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
        7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
    }

    months = []

    # If no specific period is selected, all data will be used.
    if period is None:
        period_name = 'whole_year'
        months = list(range(1, 13))  # All months
    elif isinstance(period, int):
        period_name = month_names[period]
        months = [period]
    elif isinstance(period, str):
        # Check if the input is a single season or multiple seasons
        if 'and' in period:
            seasons = period.lower().split('and')
            period_name = ''
            for season in seasons:
                season = season.strip()
                months.extend(seasons_to_months.get(season, []))
                period_name += ''.join(month_names[m] for m in seasons_to_months.get(season, []))
        else:
            months = seasons_to_months.get(period.lower(), [])
            period_name = ''.join(month_names[m] for m in months)
    elif isinstance(period, list):
        period_name = ''.join(month_names[m] for m in period if m in month_names)
        months = period
    else:
        raise ValueError("Period must be None, an integer, a string representing a single season, "
                         "a string with multiple seasons separated by 'and', or a list of integers.")

    for k, ds in ds_dict_copy.items():
        if start_year and end_year:
            start_date = f'{start_year}-01-01'
            end_date = f'{end_year}-12-31'
            ds = ds.sel(time=slice(start_date, end_date))

        # If months are specified, select those months
        if months:
            month_mask = ds['time.month'].isin(months)
            ds = ds.where(month_mask, drop=True)

        # Store the original attributes of each variable
        original_attrs = {var: ds[var].attrs for var in ds.data_vars}

        # If yearly_sum is True, sum over 'time' dimension to get yearly sum
        if yearly_sum: # does only make sense for accumulative variables e.g. pr or tran
            attrs = ds.attrs
            # Multiply each value by the number of days in the respective month
            days = ds['time'].dt.days_in_month
            ds = (ds * days).resample(time='AS').sum(dim='time')
            sum_type = 'yearly_sum'
            ds.attrs = attrs
        else:
            sum_type = 'monthly_mean'

        # Reassign the original attributes back to each variable
        for var in ds.data_vars:
            ds[var].attrs = original_attrs[var]

        ds_dict_copy[k] = ds
        ds_dict_copy[k].attrs['months'] = period_name
        ds_dict_copy[k].attrs['yearly_sum'] = sum_type

    return ds_dict_copy

In [52]:
# ========= Create a helper function to open the dataset ========
def open_dataset(filename):
    ds = xr.open_dataset(filename)
    return ds

# Define a helper function to open and merge datasets
def open_and_merge_datasets(folder, model, experiment_id, temp_res, variables):
    filepaths = []
    for var in variables:
        path = f'../../data/CMIP6/{experiment_id}/{folder}/{temp_res}/{var}'
        fp = glob.glob(os.path.join(path, f'CMIP.{model}.{experiment_id}.{var}.regridded.nc'))
        if fp:
            filepaths.append(fp[0])
        else:
            print(f"No file found for variable '{var}' in model '{model}'.")
            print(fp)

    datasets = [xr.open_dataset(fp) for fp in filepaths]
    ds = xr.merge(datasets)
    return ds

In [53]:
def calculate_monthly_mean(ds, variable):
    """
    Calculate the mean for each month across all years for a given variable in the dataset.

    Parameters:
    ds (xarray.Dataset): The input dataset.
    variable (str): The name of the variable to calculate the monthly mean for.

    Returns:
    xarray.Dataset: A dataset containing the monthly mean for the specified variable.
    """
    # Calculate the monthly mean
    monthly_means = ds[variable].groupby('time.month').mean('time')

    # Create a new dataset for the monthly means
    monthly_mean_ds = xr.Dataset({variable: monthly_means})

    # Copy attributes from the original dataset and variable
    monthly_mean_ds.attrs = ds.attrs
    monthly_mean_ds[variable].attrs = ds[variable].attrs
    monthly_mean_ds[variable].attrs['description'] = f'Monthly mean of {variable}'

    return monthly_mean_ds

In [54]:
def calculate_gsl(ds_dict, season='all'):
    for name, ds in ds_dict.items():
        print(ds.source_id)
        monthly_mean_ds = calculate_monthly_mean(ds, 'lai')
        gsl_ds = calculate_growing_season_length(monthly_mean_ds, season=season)
        ds_dict[name] = gsl_ds
        
    return ds_dict

In [55]:
def calculate_growing_season_length(ds_monthly_mean, season='period'):
    lai = ds_monthly_mean['lai']

    # Create a DataArray for storing the growing season length
    growing_season_length = xr.DataArray(
        np.nan, 
        dims=('lat', 'lon'), 
        coords={'lat': ds_monthly_mean.lat, 'lon': ds_monthly_mean.lon}
    )
    
    seasons_to_months = {
        'winter': [12, 1, 2],
        'spring': [3, 4, 5],
        'summer': [6, 7, 8],
        'fall': [9, 10, 11],
        'period': range(1, 13)
    }
    
    selected_months = seasons_to_months[season]

    for lat in ds_monthly_mean.lat.values:
        for lon in ds_monthly_mean.lon.values:
            lai_ts = lai.sel(lat=lat, lon=lon)

            # Check if the LAI time series is entirely NaN (ocean cell)
            if lai_ts.isnull().all():
                continue  # Skip this cell, it remains NaN in growing_season_length

            monthly_pct_change = calculate_monthly_pct_change(lai_ts)
            starts, ends = detect_season_starts_and_ends(monthly_pct_change)
            if season == 'winter':
                length = winter_gsl_algorithm(starts, ends, selected_months)
            elif season == 'yearly':
                length = calculate_season_length(starts, ends)
            else:
                length = gsl_algorithm(starts, ends, selected_months)
            growing_season_length.loc[dict(lat=lat, lon=lon)] = length

    # Create a new dataset for the growing season length
    ds_growing_season_length = xr.Dataset()
    ds_growing_season_length[f'growing_season_length_{season}'] = growing_season_length
    ds_growing_season_length[f'growing_season_length_{season}'].attrs = {
        'description': f'Length of the growing season in months for {season}',
        'calculated_from': 'LAI',
        'season': season,
        'units': '',
        'long_name': 'Growing Season Length' 
    }
    
    ds_growing_season_length.attrs = ds_monthly_mean.attrs
    ds_growing_season_length.attrs['table_id'] = season
    

    return ds_growing_season_length

In [56]:
def winter_gsl_algorithm(starts, ends, selected_months):
     # Set december to month number 0
    # Adjust starts and ends lists if December is present, setting it to month number 0
    # and ensuring selected_months reflects winter months [0, 1, 2]
    starts = [0 if month == 12 else month for month in starts]
    ends = [0 if month == 12 else month for month in ends]

    # If any start or end was adjusted to 0, indicating December was present,
    # then set selected_months for winter
    if 0 in starts or 0 in ends:
        selected_months = [0, 1, 2]
    
    total_growing_season_length = 0
    
    for start, end in zip(starts, ends): 
        # Start in SM & End in SM
        if start in selected_months and end in selected_months:
            if start < end:
                growing_season_length = end - start
            elif start > end:
                if start == 1:
                    growing_season_length = 2
                else:
                    growing_season_length = (2 - start + 1) + (end)
        # Start in SM & End not
        elif start in selected_months and end not in selected_months:
            position = selected_months.index(start)
            growing_season_length = len(selected_months) - (position)
        # End in SM & Start not
        elif start not in selected_months and end in selected_months:
            selected_months.reverse()
            position = selected_months.index(end)
            growing_season_length = max(0, len(selected_months) - position - 1)
            selected_months.reverse()
        # Start not in SM & End not in SM
        elif start not in selected_months and end not in selected_months:
            if start <= end: 
                growing_season_length = 0
            elif start > end: 
                growing_season_length = 3
        # Combination missing
        else:
            raise ValueError("The start and end combination for the growing season is missing or invalid.")
        
        total_growing_season_length += growing_season_length

    # Revert starts and ends from 0 to 12 if they were previously adjusted to represent December
    starts = [12 if month == 0 else month for month in starts]
    ends = [12 if month == 0 else month for month in ends]

    # If December (12) is now present in starts or ends, adjust selected_months for winter
    if 12 in starts or 12 in ends:
        selected_months = [12, 1, 2]
    
    return total_growing_season_length

In [57]:
def gsl_algorithm(starts, ends, selected_months):
    
    total_growing_season_length = 0
    
    for start, end in zip(starts, ends):
        # Start in SM & End in SM
        if start in selected_months and end in selected_months:
            if start < end:
                growing_season_length = end - start
            elif start > end:
                growing_season_length = (2 - start + 1) + (end)
        # Start in SM & End not
        elif start in selected_months and end not in selected_months:
            position = selected_months.index(start)
            growing_season_length = len(selected_months) - (position)
        # End in SM & Start not
        elif start not in selected_months and end in selected_months:
            selected_months.reverse()
            position = selected_months.index(end)
            growing_season_length = max(0, len(selected_months) - position - 1)
            selected_months.reverse()
        # Start not in SM & End not in SM
        elif start not in selected_months and end not in selected_months:
            if start == end: 
                growing_season_length = 0
            if start < end or start > end: 
                if end < min(selected_months):
                    growing_season_length = 0
                elif end > max(selected_months) and start > max(selected_months):
                    growing_season_length = 0
                else:
                    growing_season_length = 3
        else:
            raise ValueError("The start and end combination for the growing season is missing or invalid.")
            
        total_growing_season_length += growing_season_length
        
    return total_growing_season_length

In [81]:
def cyclic_distance(start, end):
    """Calculate the cyclic distance from start to end considering a wrap around from 12 to 1."""
    if end >= start:
        return end - start
    else:
        return 12 - start + end

def reorder_ends(starts, ends, monthly_pct_change):
    # Ensure inputs are lists of equal length
    if len(starts) != len(ends):
        print(starts, ends)
        print(monthly_pct_change.lat.values, monthly_pct_change.lon.values)
        raise ValueError("Starts and ends must be of equal length.")
    
    reordered_ends = []  # To store the reordered ends
    available_ends = ends.copy()  # Copy ends to manipulate

    for start in starts:
        # Calculate cyclic distances for each available end
        distances = [(end, cyclic_distance(start, end)) for end in available_ends]
        
        # Find the end with the minimum positive cyclic distance to the start
        closest_end = min(distances, key=lambda x: x[1])[0]
        
        reordered_ends.append(closest_end)
        available_ends.remove(closest_end)  # Remove the selected end

    return reordered_ends

In [95]:
def handle_inf_values(monthly_pct_change):
    for i in range(len(monthly_pct_change)):
        if np.isinf(monthly_pct_change[i]):
            # Find the previous finite change
            prev_finite_index = i - 1
            while prev_finite_index >= 0 and (np.isinf(monthly_pct_change[prev_finite_index]) or np.isnan(monthly_pct_change[prev_finite_index])):
                prev_finite_index -= 1
            
            # Ensure loop wraps around if necessary
            if prev_finite_index < 0:
                prev_finite_index = len(monthly_pct_change) - 1
                while np.isinf(monthly_pct_change[prev_finite_index]) or np.isnan(monthly_pct_change[prev_finite_index]):
                    prev_finite_index -= 1
                    if prev_finite_index < 0:
                        # If all values are inf or NaN, break to avoid infinite loop
                        return monthly_pct_change

            # Find the next finite change
            next_finite_index = (i + 1) % len(monthly_pct_change)
            while next_finite_index != i and (np.isinf(monthly_pct_change[next_finite_index]) or np.isnan(monthly_pct_change[next_finite_index])):
                next_finite_index = (next_finite_index + 1) % len(monthly_pct_change)
            
            # Set the inf value to the midpoint of the changes before and after
            if not np.isinf(monthly_pct_change[prev_finite_index]) and not np.isinf(monthly_pct_change[next_finite_index]):
                monthly_pct_change[i] = (monthly_pct_change[prev_finite_index] + monthly_pct_change[next_finite_index]) / 2
            else:
                # Fallback if no suitable values were found
                monthly_pct_change[i] = 0

    return monthly_pct_change

def calculate_monthly_pct_change(lai_ts):
    monthly_pct_change = ((lai_ts - lai_ts.roll(month=1)) / lai_ts.roll(month=1)) * 100
    monthly_pct_change[0] = ((lai_ts[0] - lai_ts[-1]) / lai_ts[-1]) * 100
    
    # Replace NaN values with 0
    monthly_pct_change = np.where(np.isnan(monthly_pct_change), 0, monthly_pct_change)
    
    # Handle inf values separately
    monthly_pct_change = handle_inf_values(monthly_pct_change)
    
    return monthly_pct_change

In [94]:
def calculate_directional_change(lai_ts):
    # Initialize an array of zeros with the same shape as lai_ts to store our results
    directional_change = np.zeros_like(lai_ts)
    
    # Roll the array to compare each month to the previous month
    prev_month_lai = np.roll(lai_ts, shift=1)
    
    # Calculate the change, avoiding division entirely
    raw_change = lai_ts - prev_month_lai
    
    # Determine the direction of change (+1 for increase, -1 for decrease, 0 for no change)
    directional_change = np.sign(raw_change)
    
    # Handling the first element manually since it compares the first and last elements
    # If this behavior is not desired, set the first element to 0 or another placeholder
    directional_change[0] = 0  # Or np.nan if you prefer it to be explicitly undefined
    
    return directional_change 

In [92]:
def detect_season_starts_and_ends(monthly_pct_change):
    starts, ends = [], []
    
    # Check if December starts or ends a growing season 
    # STARTS
    # Wenn November kleiner gleich 0 und December größer 0 beginnt die GS
    if monthly_pct_change[-2] <= 0 and monthly_pct_change[-1] > 0:
        starts.append(12) 
    # ENDS
    # Wenn November größer 0 und December kleiner gleich 0 endet die GS
    if monthly_pct_change[-2] > 0 and monthly_pct_change[-1] <= 0:
        ends.append(12)
    
    #Check if January stats or ends a growing season
    # STARTS
    # Wenn Dezember kleiner gleich 0 und Januar größer Null beginnt die GS
    if monthly_pct_change[-1] <= 0 and monthly_pct_change[0] > 0:
        starts.append(1) 
    # ENDS
    # Wenn Dezember größer 0 und Januar kleiner gleich 0
    if monthly_pct_change[-1] > 0 and monthly_pct_change[0] <= 0:
        ends.append(1)
        
    # Now handle February to November for starts and ends
    for i in range(1, len(monthly_pct_change) - 1):
        if monthly_pct_change[i-1] <= 0 and monthly_pct_change[i] > 0:
            starts.append(i + 1)
        elif monthly_pct_change[i-1] > 0 and monthly_pct_change[i] <= 0:
            ends.append(i + 1)
    
    if len(starts) > 1 and len(ends) > 1:
        ends = reorder_ends(starts, ends, monthly_pct_change)
            
    return starts, ends

def calculate_season_length(starts, ends):
    growing_season_length = 0
    
    # Calculate the growing season length(s)
    if len(starts) > 1:
        if any(end > starts[0] for end in ends):
            closest_end = min(filter(lambda end: end > starts[0], ends), key=lambda end: end - starts[0], default=None)
            growing_season_length = closest_end - starts[0]
        else:
            growing_season_length = min(ends) + (12 - starts[0])

        if any(end > starts[1] for end in ends):
            growing_season_length = growing_season_length + max(ends) - starts[1]
        else:
            growing_season_length = growing_season_length + min(ends) + (12 - starts[1])
    elif len(starts) == 1:
        if starts[0] < ends[0]:
            growing_season_length = ends[0] - starts[0]
        elif starts[0] > ends[0]:
            growing_season_length = (12 - starts[0]) + ends[0]
    
    return growing_season_length

In [60]:
def save_file(save_file, folder, save_var=True):
    """
    Save files as netCDF.

    Args:
        savefile (dict or dataset): Dictionary of xarray datasets or dataset.
        folder (string): Name of folder data is saved in.
        save_var (boolean): If True, data is saved separately for each variable. If false, one file is saved with all variables.
        

    Returns:
        nc_out: Path were data is saved in.
    """
    
    if save_var:
        for key, ds in ds_dict.items():
            for var in ds:
                # Variable to keep
                variable_to_keep = var
                dimensions_to_keep = {'time', 'lat', 'lon'}
                coordinates_to_keep = {'time', 'lat', 'lon'}

                if any('depth' in ds[var].dims for var in ds.variables):
                    dimensions_to_keep.add('depth')
                    coordinates_to_keep.add('depth')

                # Create a new dataset with only the desired variable
                ds_var = ds[[variable_to_keep]]

                # Keep only the desired dimensions
                ds_var = ds_var.isel({dim: slice(None) for dim in dimensions_to_keep.intersection(ds_var.dims)})

                # Set the desired coordinates
                coords_to_set = set(ds_var.variables).intersection(coordinates_to_keep)
                ds_var = ds_var.set_coords(list(coords_to_set))

                savepath = f'../../data/CMIP6/{ds_var.experiment_id}/{folder}/{ds_var.table_id}/{var}/'
                filename = f'CMIP.{ds_var.source_id}.{ds_var.experiment_id}.{var}.regridded.nc'
                nc_out = os.path.join(savepath, filename)
                os.makedirs(savepath, exist_ok=True) 
                if os.path.exists(nc_out):
                    os.remove(nc_out)
                    print(f"File  with path: {nc_out} removed")
                    
                # Save to netcdf file
                with dask.config.set(scheduler='threads'):
                    ds_var.to_netcdf(nc_out)
                    print(f"File with path: {nc_out} saved")

    return nc_out

### 1. Load netCDF files

In [96]:
# ========= Define period, models and path ==============
variables=['lai']
folder='preprocessed'
temp_res = 'month'

# ========= Use Dask to parallelize computations ==========
dask.config.set(scheduler='processes')

# Create dictionary using a dictionary comprehension and Dask
experiment_id = 'ssp370'
source_id = ['CESM2-WACCM'] #['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CanESM5-CanOE', 'CanESM5', 'CESM2-WACCM','CNRM-CM6-1', 'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','MIROC-ES2L', 'MPI-ESM1-2-LR','NorESM2-MM','TaiESM1','UKESM1-0-LL']
ds_dict = dask.compute({model: open_and_merge_datasets(folder, model, experiment_id, temp_res, variables) for model in source_id})[0]

#experiment_id = 'ssp126'
#source_id = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CanESM5-CanOE', 'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']
#ds_dict_ssp126 = dask.compute({model: open_and_merge_datasets(folder, model, experiment_id, temp_res, variables) for model in source_id})[0]

In [62]:
# ========= Have a look into the dictionary =======
print(list(ds_dict.keys()))
ds_dict[list(ds_dict.keys())[0]].table_id

['CESM2-WACCM']


'Amon'

In [97]:
ds_dict = select_period(ds_dict, start_year=2071, end_year=2100)

### Compute GSL

In [ ]:
ds_dict = calculate_gsl(ds_dict, 'period') # period, winter, spring, summer, fall

CESM2-WACCM


In [ ]:
ds_dict[list(ds_dict.keys())[0]].growing_season_length_yearly.plot()

### Save files

In [ ]:
nc_out = save_file(ds_dict, folder=folder)

In [ ]:
#test if data is correct
xr.open_dataset(nc_out)